In [3]:
# import necessary modules
import raadpy as rp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import to_hex
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import pandas as pd
import os
import csv

%matplotlib widget

In [22]:
# The functions:

# Parse a command (command and time)
def desc_finder(line:str,cmdlist,outputs,i,time,failed_idx):
    """Parse a command and return its status and description

    Args:
        line (str): The string of the command
        cmdlist (pd.DataFrame): pandas data frame with the commands and their equivalent messages
        outputs (_type_): _description_
        i (_type_): _description_
        time (_type_): _description_
        failed_idx (_type_): _description_

    Returns:
        _type_: _description_
    """

    # Get the description
    status = 1
    splt = line.split(' ')

    # define end of log file
    if splt[-1] == 'SE0>':
        desc = 'LOG END'
        desc = [-1,17]
        text = "LOG END"

    # define commands from the command file
    elif 'txrx' in splt[1]:
        node,port,msg = int(splt[2]),int(splt[3]),str(splt[5])

        index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port) & (cmdlist['Message'].str.startswith(msg)),['ID_COMMAND_Proposed','ID_in_Graph']]
        
        # include the power shutdown
        if node == 4:
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID_COMMAND_Proposed','ID_in_Graph']]

        # include the custom scenario
        if port == 9:
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID_COMMAND_Proposed','ID_in_Graph']]

        # include the mysterious command
        if msg in "0F":
            index = cmdlist.loc[(cmdlist['NODE']==node) & (cmdlist['PORT']==port),['ID_COMMAND_Proposed','ID_in_Graph']]

        

        #If index did not find anything
        if len(index) == 0:
            text = ("")
            desc = ("not a payload cmnd ",line)
            
        # if command found in command list
        else:
            desc = list(index.to_numpy()[0])
            text = ("cmnd found in list ",line)
        

        if i in [fid for fid in failed_idx]:
            time = time + (float(splt[4])/1000)
            status = -1
        

    elif splt[1] == 'delay':
        dd   = (float(splt[2])/1000)
        time = time + dd
        ld   =  ("delay of ", dd )
        desc = (ld, line)
        text = ("")

    elif splt[1] == 'delayuntil':
        time = float(splt[2])
        ld =  ("delay until ", time)
        desc = (ld, line)
        text = ("") 

    elif 'read' in splt[1]:
        time = float(outputs[i][0].split(' ')[3])
        desc = ("read the current time ", time, line)
        text = ("")

    else:
        desc = ("unidentified command " , line)
        text = ("")
    
    return desc,time,status,text


# Decode a logfile by explaning every command and pinpointing when it happened
def decode_log(filename:str="../Data/Light-1/light1-2022-09-15/light1-2022-09-15-se-log.txt"):
    """Take a log file and parse its commands to uncover thier timestamps

    Args:
        filename (str, optional): The filepath and filename of the log file. Defaults to "../Data/Light-1/light1-2022-09-15/light1-2022-09-15-se-log.txt".

    Returns:
        _type_: Log, commands, outputs, description, failed_idx, loglines_array
    """
    # Identify Date of file:
    date = filename.split('/')[-2][7:]

    # Load the logfile
    logfile = open(filename)
    cmdlist = pd.read_csv("command_list_modified.csv")

    # Load the lines
    loglines = logfile.readlines()

    # Close the file
    logfile.close()

    # Create an array with the lines
    commands    = []
    outputs     = []
    idx         = []
    description = []
    times       = []
    fails       = []
    failed_idx  = []
    text_desc   = []

    # Get commands and their indices
    for i, line in enumerate(loglines):
        if 'SE0>' in line:
            commands.append(line)
            idx.append(i)

    # Check if the last command was empty
    if loglines[-1] != 'SE0>':
        commands.append('SE0>')
        idx.append(len(loglines))

    # Get the command output
    for i in range(len(idx)-1):
        out = []
        if 'SE0>#' not in commands[i]:
            for j in range(idx[i]+1,idx[i+1]):
                out.append(loglines[j])
        
        outputs.append(out)

    # Create the dictionary
    log = [[command,output] for command,output in zip(commands,outputs)]

    # Find the commands that failed
    for i in range(len(commands)-1):
        for output in log[i][1]:
            if 'FAIL' in output:
                failed_idx.append(i)

    # Initialize time at 0s
    time = 0
    
    # Give commands and use the finder function to pull the description, time ran, and if the command failed.
    for k, id in enumerate(idx):
        c = commands[k]
        cmd = c.split('\n')[0]
        desc,time,failed,text = desc_finder(cmd,cmdlist,outputs,k,time,failed_idx)

        if len(desc) != 0:
            description.append(desc)
            times.append(time)
            fails.append(failed)
            text_desc.append(text)
    decoded_file_name = ("Decoded_Logfile_" + date + ".csv")
    fileDir = os.path.join('Defined_Logs')

    if not os.path.exists(fileDir):
        os.makedirs(fileDir)
    
    filePath = os.path.join(fileDir, decoded_file_name)

    #Name the headers of the CSV File
    # header = ['status','time','description','ID_in_Graph']
    header = ["Epoch Time","Detector: PMT or SiPM","CMND #","CMND Description"]

    loglines_array = []
    pmt_cmds  = [1,2,3,4,5,6,7,8,9,10,11,23,25,27,29,31,33,34,36] 
    sipm_cmds = [12,13,14,15,16,17,18,19,20,21,22,24,26,28,30,32,33,35,37]

    #Clear the file and write from scratch
    with open(filePath, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

        for i in range(len(description)):
            if fails[i] != -1: #Only show commands that were executed successfully
                
                
                #Only show the payload cmnds, no time reading cmnds
                #Sort cmnds by detector:
                g = (times[i],)
                loglines_array.append(g)
                writer.writerow(g)

    # Return everything else
    return log, commands[:-1], outputs, description, failed_idx, loglines_array


In [23]:
# Choose date and run the code:

Date= "2022-09-15"
a,a,a,a,a,loglines_array = decode_log("../Data/Light-1/light1-" + Date + "/light1-" + Date + "-se-log.txt")
